# Introduction

In this notebook, we are going to clean a real-world dataset together.

The dataset consists of scraped salad-related data that needs cleaning before it can be used for analysis.  
We will go through the data cleaning process step by step, sharing the mindset and reasoning behind each decision we make along the way.

The main goal is to prepare the dataset for future tasks, such as exploration, visualization, or modeling.

**Prerequisites:**

- Intermediate Python knowledge  
- Prior experience with Python libraries such as **Pandas** and **NumPy**

📺 For a full walkthrough of the cleaning process with detailed explanations, I’ve made a [YouTube video](https://www.youtube.com/channel/UChd_ESjPkUx1r7nY8b8rLGA) accompanying this notebook.


In [226]:
#importing the necessary libraries
import pandas as pd
import numpy as np

### let's import and check the five few rows of our data

In [227]:
data = pd.read_csv('messy_salad_data.csv')
data.head()

,salad_title,prep_time,n_star,n_review,Calories,Total Fat,Saturated Fat,Carbohydrates,Dietary Fiber,Sugar,Protein,Cholesterol,Sodium
0,Antipasto Salad,35 minutes,5 of 5 stars,190 Reviews,['647 '],['37 g'],['8 g'],['58 g'],['3 g'],['2 g'],['20 g'],['34 mg'],['433 mg']
1,Shrimp Salad,30 minutes,5 of 5 stars,175 Reviews,['407 '],['31 g'],['5 g'],['2 g'],['1 g'],['1 g'],['31 g'],['258 mg'],['536 mg']
2,Chicken Salad,30 minutes,4.5 of 5 stars,277 Reviews,['1134 '],['80 g'],['17 g'],['19 g'],['2 g'],['7 g'],['80 g'],['250 mg'],['1870 mg']
3,Broccoli Salad,1 hour 15 minutes,5 of 5 stars,170 Reviews,['316 '],['27 g'],['6 g'],['16 g'],['0 g'],['12 g'],['4 g'],['24 mg'],['292 mg']
4,Cafe Green Salad,10 minutes,5 of 5 stars,297 Reviews,['10 grams '],['1.5 grams '],['0 milligrams '],['156 milligrams '],['2 grams '],['1 grams '],['1 grams '],['1 grams '],['103 calorie ']


In [228]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   salad_title    230 non-null    object
 1   prep_time      230 non-null    object
 2   n_star         230 non-null    object
 3   n_review       230 non-null    object
 4   Calories       220 non-null    object
 5   Total Fat      220 non-null    object
 6   Saturated Fat  220 non-null    object
 7   Carbohydrates  220 non-null    object
 8   Dietary Fiber  220 non-null    object
 9   Sugar          220 non-null    object
 10  Protein        220 non-null    object
 11  Cholesterol    220 non-null    object
 12  Sodium         220 non-null    object
dtypes: object(13)
memory usage: 23.5+ KB


We need to fix the data types and handle the missing values.

### Dealing with the Preparation Time Column

In [229]:
data.prep_time

0              35 minutes
1              30 minutes
2              30 minutes
3       1 hour 15 minutes
4              10 minutes
              ...        
225            30 minutes
226    6 hours 10 minutes
227            10 minutes
228            15 minutes
229            10 minutes
Name: prep_time, Length: 230, dtype: object

In [230]:
for a in data.prep_time:
    if 'm' not in a:
        print(a)

1 hour
4 hours
1 hour
Unknown
Unknown
1 hour
Unknown
3 hours
1 hour
Unknown
1 hour
1 hour
1 hour
Unknown
Unknown
1 hour
Unknown
Unknown


As we can see here, we also have unknown values. We need to create a function that can handle all of them and convert the entire column into minutes.

In [231]:
def duration_to_minutes(sample):
    if sample.lower() =='unknown':
        return 0
    total_minutes = 0
    parts = sample.split()
    
    if len(parts) ==2 and  'h' in parts[1].lower():
        total_minutes += int(parts[0]) * 60
    elif len(parts) ==2 and  'm' in parts[1].lower():
        total_minutes += int(parts[0]) 
    elif len(parts)==4 and ('h' in parts[1] and 'm' in parts[3]):
        total_minutes += int(parts[0])*60 + int(parts[2])
    else:
        total_minutes
    return total_minutes

In [232]:
data['prep_time'] = data['prep_time'].apply(duration_to_minutes)
data['prep_time'] = data['prep_time'].astype('int')

In [233]:
data['prep_time'].isnull().sum()

0

know we have fixed the prep_time column

In [234]:
data.head(1)

,salad_title,prep_time,n_star,n_review,Calories,Total Fat,Saturated Fat,Carbohydrates,Dietary Fiber,Sugar,Protein,Cholesterol,Sodium
0,Antipasto Salad,35,5 of 5 stars,190 Reviews,['647 '],['37 g'],['8 g'],['58 g'],['3 g'],['2 g'],['20 g'],['34 mg'],['433 mg']


now let's fix the star and review columns

In [235]:
for a in data.n_star:
    if 's' not in a:
        print(a)

Unknown
Unknown
Unknown


In [236]:
for a in data.n_review:
    if 'R' not in a:
        print(a)

no review info
no review info
no review info


In [237]:
# let's create a function that handles both cases
def extracting_num(sample):
    parts = sample.split()
    if parts[0].lower() in {"unknown", "no"}:
        return None
    else:
        value = parts[0]
    return value

In [238]:
data['n_star'] = data['n_star'].apply(extracting_num)
data['n_review'] = data['n_review'].apply(extracting_num)

In [239]:
print(data[data['n_star'].isna()]),
print(data[data['n_review'].isnull()])

    salad_title  prep_time n_star n_review Calories Total Fat Saturated Fat  \
62      Unknown          0   None     None      NaN       NaN           NaN   
183     Unknown          0   None     None      NaN       NaN           NaN   
211     Unknown          0   None     None      NaN       NaN           NaN   

    Carbohydrates Dietary Fiber Sugar Protein Cholesterol Sodium  
62            NaN           NaN   NaN     NaN         NaN    NaN  
183           NaN           NaN   NaN     NaN         NaN    NaN  
211           NaN           NaN   NaN     NaN         NaN    NaN  
    salad_title  prep_time n_star n_review Calories Total Fat Saturated Fat  \
62      Unknown          0   None     None      NaN       NaN           NaN   
183     Unknown          0   None     None      NaN       NaN           NaN   
211     Unknown          0   None     None      NaN       NaN           NaN   

    Carbohydrates Dietary Fiber Sugar Protein Cholesterol Sodium  
62            NaN           NaN

as we can see here, we have NaN values, in this case, we are going to delete them because
the entire rows are missing.

In [240]:
data.dropna(subset=['n_star', 'n_review'], inplace=True)

In [241]:
print(data[data['n_star'].isna()]),
print(data[data['n_review'].isnull()])

Empty DataFrame
Columns: [salad_title, prep_time, n_star, n_review, Calories, Total Fat, Saturated Fat, Carbohydrates, Dietary Fiber, Sugar, Protein, Cholesterol, Sodium]
Index: []
Empty DataFrame
Columns: [salad_title, prep_time, n_star, n_review, Calories, Total Fat, Saturated Fat, Carbohydrates, Dietary Fiber, Sugar, Protein, Cholesterol, Sodium]
Index: []


In [242]:
# fixing the data type
data['n_review'] = data['n_review'].astype('int')
data['n_star'] = data['n_star'].astype('float')

In [243]:
data.head(3)

,salad_title,prep_time,n_star,n_review,Calories,Total Fat,Saturated Fat,Carbohydrates,Dietary Fiber,Sugar,Protein,Cholesterol,Sodium
0,Antipasto Salad,35,5.0,190,['647 '],['37 g'],['8 g'],['58 g'],['3 g'],['2 g'],['20 g'],['34 mg'],['433 mg']
1,Shrimp Salad,30,5.0,175,['407 '],['31 g'],['5 g'],['2 g'],['1 g'],['1 g'],['31 g'],['258 mg'],['536 mg']
2,Chicken Salad,30,4.5,277,['1134 '],['80 g'],['17 g'],['19 g'],['2 g'],['7 g'],['80 g'],['250 mg'],['1870 mg']


In [244]:
# let's remove [] '' from the entire data
data = data.apply(lambda x: x.str.replace('[','').str.replace(']','').str.replace("'","") if x.dtype =='object' else x)

In [245]:
data.head(3)

,salad_title,prep_time,n_star,n_review,Calories,Total Fat,Saturated Fat,Carbohydrates,Dietary Fiber,Sugar,Protein,Cholesterol,Sodium
0,Antipasto Salad,35,5.0,190,647,37 g,8 g,58 g,3 g,2 g,20 g,34 mg,433 mg
1,Shrimp Salad,30,5.0,175,407,31 g,5 g,2 g,1 g,1 g,31 g,258 mg,536 mg
2,Chicken Salad,30,4.5,277,1134,80 g,17 g,19 g,2 g,7 g,80 g,250 mg,1870 mg


In [246]:
data[data.isnull().any(axis=1)]

,salad_title,prep_time,n_star,n_review,Calories,Total Fat,Saturated Fat,Carbohydrates,Dietary Fiber,Sugar,Protein,Cholesterol,Sodium
21,New Potato Salad,240,5.0,133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,Vinaigrette For Green Salad,15,5.0,137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,Big Steak Salad,35,5.0,61,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
128,Chilled Asparagus Salad,75,5.0,46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
153,Cucumber Salad,0,4.5,38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
161,Buffalo Chicken Salad,35,5.0,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178,Avocado Salad,15,5.0,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [247]:
data = data.dropna()

In [248]:
data[data['prep_time']==0]

,salad_title,prep_time,n_star,n_review,Calories,Total Fat,Saturated Fat,Carbohydrates,Dietary Fiber,Sugar,Protein,Cholesterol,Sodium
44,Chicken Salad Contessa,0,4.5,188,1420,110 g,19 g,26 g,6 g,15 g,86 g,305 mg,1393 mg
93,Cape Cod Chopped Salad,0,5.0,121,749,65 g,17 g,29 g,4 g,22 g,17 g,61 mg,1000 mg
107,"Arugula, Watermelon and Feta Salad",0,5.0,148,685,47 g,15 g,58 g,5 g,42 g,17 g,76 mg,1460 mg
139,Barefoot Carrot Salad,0,4.5,87,341,19 g,4 g,44 g,5 g,33 g,3 g,19 mg,574 mg


There are 4 missing values in the preparation time column. Let's fill them in.

In [249]:
# let's create a temporary dataset without those salads
temp_data = data.drop([44,93,107,139])

In [250]:
temp_data['prep_time'].describe()

count    216.000000
mean      46.310185
std       56.090302
min        5.000000
25%       20.000000
50%       30.000000
75%       50.000000
max      490.000000
Name: prep_time, dtype: float64

In [251]:
chicken = temp_data[temp_data['salad_title'].str.contains('chicken', case = False)]
chicken.prep_time.describe()

count     20.000000
mean      57.650000
std       38.788597
min       16.000000
25%       33.000000
50%       52.500000
75%       71.250000
max      195.000000
Name: prep_time, dtype: float64

Salads with 'chicken' in the title take, on average, 12 minutes longer to prepare.

Let's fill each missing salad preparation time based on whether 'chicken' is present in the title.



In [252]:
data.loc[data['salad_title']=='Chicken Salad Contessa','prep_time'] =58
data.loc[data.index.isin([93, 107, 139]), 'prep_time'] = 47

In [253]:
# we have misslabeling issues, some Sodium values contains calories.
data['Sodium']

0            433 mg
1            536 mg
2           1870 mg
3            292 mg
4      103 calorie 
           ...     
225         1110 mg
226          843 mg
227          252 mg
228          257 mg
229          234 mg
Name: Sodium, Length: 220, dtype: object

In [254]:
data['Calories']

0           647 
1           407 
2          1134 
3           316 
4      10 grams 
         ...    
225         954 
226         163 
227         127 
228         303 
229         165 
Name: Calories, Length: 220, dtype: object

In [255]:
# let's create a function that shifts values
def shift_values(row):
    cal_parts = row['Calories'].split()
    sodium_parts = row['Sodium'].split()
    if len(cal_parts) > 1 and len(sodium_parts) > 1:

        if 'g' in cal_parts[1].lower() and 'c' in sodium_parts[1].lower():
            row['Calories'], row['Sodium'] = row['Sodium'], row['Calories']
    return row

In [256]:
data = data.apply(shift_values, axis = 1)

In [257]:
data.head()

,salad_title,prep_time,n_star,n_review,Calories,Total Fat,Saturated Fat,Carbohydrates,Dietary Fiber,Sugar,Protein,Cholesterol,Sodium
0,Antipasto Salad,35,5.0,190,647,37 g,8 g,58 g,3 g,2 g,20 g,34 mg,433 mg
1,Shrimp Salad,30,5.0,175,407,31 g,5 g,2 g,1 g,1 g,31 g,258 mg,536 mg
2,Chicken Salad,30,4.5,277,1134,80 g,17 g,19 g,2 g,7 g,80 g,250 mg,1870 mg
3,Broccoli Salad,75,5.0,170,316,27 g,6 g,16 g,0 g,12 g,4 g,24 mg,292 mg
4,Cafe Green Salad,10,5.0,297,103 calorie,1.5 grams,0 milligrams,156 milligrams,2 grams,1 grams,1 grams,1 grams,10 grams


In [258]:
data['Calories'] = data['Calories'].str.split().str[0].astype('float').astype('int')


we are soo close, we have to fix the unit know, since we have differents units like mg and g

In [259]:
def fixing_unit(value):
    parts = value.split()
    if len(parts) ==2 and (parts[1].lower() == 'g' or parts[1].lower()=='gram' or parts[1].lower()=='grams'):
        return "{:.1f}".format(float(parts[0]))
    else:
        return "{:.1f}".format(float(parts[0]) * 0.001)

In [260]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 220 entries, 0 to 229
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   salad_title    220 non-null    object 
 1   prep_time      220 non-null    int64  
 2   n_star         220 non-null    float64
 3   n_review       220 non-null    int64  
 4   Calories       220 non-null    int32  
 5   Total Fat      220 non-null    object 
 6   Saturated Fat  220 non-null    object 
 7   Carbohydrates  220 non-null    object 
 8   Dietary Fiber  220 non-null    object 
 9   Sugar          220 non-null    object 
 10  Protein        220 non-null    object 
 11  Cholesterol    220 non-null    object 
 12  Sodium         220 non-null    object 
dtypes: float64(1), int32(1), int64(2), object(9)
memory usage: 31.3+ KB


In [261]:
columns = ['Total Fat','Saturated Fat','Carbohydrates','Dietary Fiber','Sugar','Protein','Cholesterol','Sodium']

for column in columns:
    data[column]=data[column].apply(fixing_unit).astype('float')

In [262]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 220 entries, 0 to 229
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   salad_title    220 non-null    object 
 1   prep_time      220 non-null    int64  
 2   n_star         220 non-null    float64
 3   n_review       220 non-null    int64  
 4   Calories       220 non-null    int32  
 5   Total Fat      220 non-null    float64
 6   Saturated Fat  220 non-null    float64
 7   Carbohydrates  220 non-null    float64
 8   Dietary Fiber  220 non-null    float64
 9   Sugar          220 non-null    float64
 10  Protein        220 non-null    float64
 11  Cholesterol    220 non-null    float64
 12  Sodium         220 non-null    float64
dtypes: float64(9), int32(1), int64(2), object(1)
memory usage: 31.3+ KB


In [263]:
#let's save the data into a csv file
data.to_csv('cleaned_salad_data.csv', index=False)